## Imports

In [39]:
import pandas as pd
import tensorflow as tf
import numpy as np

## Read Data from csv

In [6]:
dataX = pd.read_csv('../data/train.csv',delimiter='\t')
validX = pd.read_csv('../data/valid.csv',delimiter='\t')

In [14]:
dataY = pd.read_csv('../data/train_target.csv',header=None)
validY = pd.read_csv('../data/valid_target.csv',header=None)

In [7]:
dataX.head()

,cli_pl_header,cli_pl_body,cli_cont_len,srv_pl_header,srv_pl_body,srv_cont_len,aggregated_sessions,bytes,net_samples,tcp_frag,...,application_latency,cli_tx_time,load_time,server_latency,proxy,sp_healthscore,sp_req_duration,sp_is_lat,sp_error,throughput
0,667,0,0,336,143151,143151,1,155494,1,0,...,2.195,0.0,766.522,1.097,0,0,0,0,0,202.856539
1,667,0,0,336,143151,143151,1,310988,1,0,...,2.988,0.0,766.496,1.494,0,0,0,0,0,405.726840
2,667,0,0,336,143151,143151,1,310988,1,0,...,2.209,0.0,766.544,1.104,0,0,0,0,0,405.701434
3,592,0,0,238,1269,1269,1,2401,1,0,...,3.278,0.0,293.212,1.639,0,0,0,0,0,8.188614
4,592,0,0,238,1269,1269,1,4802,1,0,...,3.292,0.0,293.205,1.646,0,0,0,0,0,16.377620


In [40]:
np_valid_x = np.array(validX)
np_valid_x[:5]

array([[  1.17900000e+03,   0.00000000e+00,   0.00000000e+00,
          4.25000000e+02,   4.59500000e+03,   4.59500000e+03,
          1.00000000e+00,   1.32340000e+04,   1.00000000e+00,
          0.00000000e+00,   1.40000000e+01,   5.00000000e+00,
          0.00000000e+00,   6.00000000e+00,   0.00000000e+00,
          1.00000000e+00,   0.00000000e+00,   4.00000000e+00,
          0.00000000e+00,   0.00000000e+00,   0.00000000e+00,
          2.73000000e+03,   1.17900000e+03,   1.00000000e+00,
          8.00000000e+00,   0.00000000e+00,   4.00000000e+00,
          0.00000000e+00,   0.00000000e+00,   0.00000000e+00,
          0.00000000e+00,   0.00000000e+00,   1.05040000e+04,
          5.02000000e+03,   2.00000000e+00,   6.55350000e+04,
          1.74400000e+00,   8.38848000e+06,   6.20000000e-02,
          2.26420000e+01,   0.00000000e+00,   2.45100000e+01,
          1.13210000e+01,   0.00000000e+00,   0.00000000e+00,
          0.00000000e+00,   0.00000000e+00,   0.00000000e+00,
        

In [41]:
np_train_x = np.array(dataX)
np_train_y = np.array(dataY)
np_valid_y = np.array(validY)

In [9]:
validX.head()

,cli_pl_header,cli_pl_body,cli_cont_len,srv_pl_header,srv_pl_body,srv_cont_len,aggregated_sessions,bytes,net_samples,tcp_frag,...,application_latency,cli_tx_time,load_time,server_latency,proxy,sp_healthscore,sp_req_duration,sp_is_lat,sp_error,throughput
0,1179,0,0,425,4595,4595,1,13234,1,0,...,22.642,0.000,24.510,11.321,0,0,0,0,0,539.942880
1,1179,0,0,425,4595,4595,1,6609,1,0,...,22.647,0.000,24.425,12.330,0,0,0,0,0,270.583419
2,1104,0,0,548,4595,4595,1,6723,1,0,...,23.263,0.000,401.409,11.631,1,0,0,0,0,16.748503
3,1179,0,0,425,4595,4595,1,13234,1,0,...,21.733,0.000,24.023,10.866,0,0,0,0,0,550.888732
4,2055,0,0,326,7324,0,1,10867,1,0,...,73.733,0.009,74.765,6.044,0,0,0,0,0,145.348759


In [15]:
dataY.head()

,0
0,8
1,8
2,8
3,5
4,5


In [18]:
validY.describe()

,0
count,761179.000000
mean,2.271618
std,5.751112
min,0.000000
25%,0.000000
50%,0.000000
75%,0.000000
max,42.000000


In [70]:
def get_batches(_X, _Y, _batch_size):
    for i in range(0, len(_X), _batch_size):
        y_one_hot = np.zeros(_batch_size,num_classes)
        y_one_hot[np.arange(_batch_size),_Y[i:i+_batch_size]] = 1
        yield (_X[i:i+_batch_size], y_one_hot)

In [71]:
batch_gen = get_batches(np_train_x,np_train_y,100)

In [72]:
next(batch_gen)[0]

IndexError: too many indices for array

# Build an MLP
_______________________

### params

In [16]:
lr = 0.001
train_steps = 15
batch_size = 100
display_step = 1

### Network params

In [31]:
input_dim = dataX.shape[1]
num_classes = dataY.max()[0]
num_hidden_1 = 2*input_dim
num_hidden_2 = input_dim // 2

### Graph Input

In [32]:
x = tf.placeholder(tf.float32, shape= [None,input_dim], name='X')
y = tf.placeholder(tf.float32, shape= [None], name = 'Y')
y1_hot = tf.one_hot()

### Weights and Biases

In [33]:
weights = {
    'h1' : tf.Variable(tf.random_normal(shape=[input_dim, num_hidden_1])),
    'h2' : tf.Variable(tf.random_normal(shape=[num_hidden_1, num_hidden_2])),
    'out': tf.Variable(tf.random_normal(shape=[num_hidden_2, num_classes]))
}
biases = {
    'h1' : tf.Variable(tf.random_normal(shape=num_hidden_1)),
    'h2' : tf.Variable(tf.random_normal(shape=num_hidden_2)),
    'out': tf.Variable(tf.random_normal(shape=num_classes))
}

### Create model

In [36]:
def mlp_model(_X, weights, biases):
    layer_1 = tf.matmul(_X, weights['h1']) + biases['h1']
    layer_2 = tf.matmul(layer_1, weights['h2']) + biases['h2']
    return tf.matmul(layer_2, weights['out']) + biases['out']

## Training Operations

In [37]:
pred = mlp_model(x,weights,biases)
# softmax loss
loss = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits(pred,y))
# training op
train_op = tf.train.AdamOptimizer(learning_rate=lr).minimize(loss)

## Training

In [ ]:
# initialize all variables
init_op = tf.initialize_all_variables()

with tf.Session() as sess:
    sess.run(init_op)
    # training begins here
    for step in train_steps:
        total_batches = len(np_train_x) // batch_size
        
        for i in range(total_batches):
            batchX, batchY = next(batch_gen)
            sess.run(train_op, feed_dict = {
                    x : batchX,
                    y : batchY
                }
        

In [66]:
y.get_shape

<bound method Tensor.get_shape of <tf.Tensor 'Y:0' shape=(?, 42) dtype=float32>>

In [73]:
_X = np_train_x[:10]
_Y = np_train_y[:10]

In [74]:
y_one_hot = 

array([[8],
       [8],
       [8],
       [5],
       [5],
       [5],
       [5],
       [5],
       [5],
       [5]])

In [77]:
one_hot = tf.one_hot(indices=[1], depth=num_classes)